In [1]:
import os
import math
import time
import numpy as np
import pandas as pd

import pyarrow
import fastparquet

# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, Dataset, DataLoader

# Pytorch Lightening
import pytorch_lightning as pl

# For Visualization
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline     
sns.set(color_codes=True)

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# local imports
from data_loading import load_data, SNPDataModule

In [3]:
input_file = "HO_data/HO_data_filtered/HumanOrigins2067_filtered.parquet"

In [4]:
snp_data = load_data(input_file)

In [5]:
snp_data.shape

torch.Size([160858, 2067])

In [6]:
# Verification
unique_values = np.unique(snp_data)
print("Unique values after normalization:", unique_values)  # Should show [0.0, 0.5, 1.0, 9.0]

Unique values after normalization: [0.  0.5 1.  9. ]


- _LightningDataModule_

In [7]:
# initialize DataModule
snp_data_module = SNPDataModule(input_path=input_file, batch_size=256, num_workers=1)

In [8]:
# Setup Data
snp_data_module.setup()

In [9]:
# Train DataLoader
train_loader = snp_data_module.train_dataloader()

In [10]:
# Get a batch from DataLoader
sample_batch = next(iter(train_loader))

In [11]:
print("Batch Shape:", sample_batch.shape)  # Expected: (batch_size, num_markers)
print("First 5 Samples:\n", sample_batch[:5])  # Show first 5 rows

Batch Shape: torch.Size([256, 2067])
First 5 Samples:
 tensor([[0.0000, 0.0000, 0.0000,  ..., 0.5000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.5000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 1.0000, 1.0000, 0.0000],
        [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.5000, 1.0000],
        [0.5000, 0.0000, 0.5000,  ..., 0.0000, 0.0000, 0.5000]])


- Time Embeddings

In [13]:
from model import SinusoidalTimeEmbedding

NameError: name 'Sequence' is not defined

## _Test Model_

In [ ]:
# create the model
diffusion_process = DiscreteDDPMProcess(num_diffusion_timesteps=1000)
time_sampler = UniformDiscreteTimeSampler(diffusion_process.tmin, diffusion_process.tmax)
model = DiffusionModel(diffusion_process, time_sampler, net_config=NetConfig(), data_shape=(6,))